In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pit

#Мы считали все таблицы, что даны в задании
train_final = pd.read_csv('train_final.csv')
test_final = pd.read_csv('test_final.csv')
#Дальше мы проверили есть ли там пустые значения (их нет)
train_final.isnull()
test_final.isnull()
#Проверяем типы данных в таблицах
train_final #Тут есть целые числа, строки, отдельные буквы, дробные числа, а также даты в формате хххх-хх-хх
test_final #Тут тоже самое, что и в прошлом 
#Вывод - В файле train_final есть столбец - is_canceled. Очевидно он для нас самый главный, ведь нам как раз надо проверить, отменит пользователь бронь или нет
#В файле test_final такого столбца нет, так как это тестовый датасет и в нём нет ответа. Нам нужно самим сказать будет отменена бронь или нет

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,City Hotel,73,2016,July,28,6,0,2,1,0.0,...,A,A,0,No Deposit,0,Transient,107.10,0,0,2016-06-12
1,City Hotel,37,2015,October,43,24,2,5,1,1.0,...,A,A,0,No Deposit,0,Transient,87.78,0,0,2015-10-31
2,City Hotel,190,2017,April,14,6,2,3,2,0.0,...,A,A,0,No Deposit,0,Transient,88.40,0,0,2016-09-29
3,City Hotel,287,2016,August,35,24,1,4,2,0.0,...,B,B,1,No Deposit,0,Transient,76.71,0,0,2016-08-29
4,Resort Hotel,386,2016,October,43,20,1,3,2,0.0,...,A,A,0,No Deposit,0,Transient-Party,49.00,0,0,2016-09-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23520,City Hotel,180,2016,May,22,23,1,5,2,0.0,...,A,A,0,No Deposit,0,Transient,80.75,0,0,2016-05-29
23521,City Hotel,4,2016,November,45,1,0,3,1,0.0,...,A,A,0,No Deposit,0,Transient,65.00,0,1,2016-11-04
23522,City Hotel,3,2016,May,21,16,1,1,1,0.0,...,A,A,0,No Deposit,0,Transient,65.00,0,0,2016-05-18
23523,Resort Hotel,312,2017,March,10,5,2,5,2,0.0,...,A,A,0,No Deposit,0,Transient-Party,30.00,0,0,2017-01-11
